<center> 
     <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300"> 
</center>

# Assignment: Notebook for Peer Assignment
Estimated time needed: 45 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets 
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions 

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01">Bank of Canada daily average exchange rates</a>


### 1. Canadian Principal Crops Data *

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701 
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.  

### 3. Bank of Canada daily average exchange rates *

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

( * these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

  1.  Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv 

  2.  Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv
  
  3.  Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv
  
  4.  Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv
  

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.
Let's first load the RODBC package:


In [1]:
library(RODBC);

## Problem 1
#### Create tables
Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R. 
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**  

The previous practice lab will help you accomplish this.


### Solution 1


In [2]:
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            # e.g. "bludb"
dsn_hostname <- "3883e7e4-18f5-4afe-be8c-fa31c41761d2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud" # e.g "54a2f15b-5c0f-46df-8954-.databases.appdomain.cloud"
dsn_port <- "31498" # e.g. "32733" 
dsn_protocol <- "TCPIP"            # i.e. "TCPIP"
dsn_uid <- "nyx77062"        # e.g. "zjh17769"
dsn_pwd <- "t0CnvOUqshCZrS1C"      # e.g. "zcwd4+8gbq9bm5k4"  
dsn_security <- "ssl"

In [3]:
conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
conn <- odbcDriverConnect(conn_path, believeNRows=FALSE)
conn

RODBC Connection 1
Details:
  case=nochange
  DRIVER={IBM DB2 ODBC DRIVER}
  UID=nyx77062
  PWD=******
  DATABASE=bludb
  HOSTNAME=3883e7e4-18f5-4afe-be8c-fa31c41761d2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
  PORT=31498
  PROTOCOL=TCPIP
  SECURITY=SSL

In [4]:
# CROP_DATA:
 crop_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv', colClasses=c(YEAR="character"))  
    head(crop_df)

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<chr>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


In [12]:
# FARM_PRICES:
farm_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv', colClasses=c(date="character"))
 head(farm_df)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“not all columns named in 'colClasses' exist”

,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<fct>,<fct>,<fct>,<dbl>
1,0,1985-01-01,Barley,Alberta,127.39
2,1,1985-01-01,Barley,Saskatchewan,121.38
3,2,1985-01-01,Canola,Alberta,342.00
4,3,1985-01-01,Canola,Saskatchewan,339.82
5,4,1985-01-01,Rye,Alberta,100.77
6,5,1985-01-01,Rye,Saskatchewan,109.75


In [7]:
# DAILY_FX:
dfx_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv', colClasses=c(DATE="character"))
 head(dfx_df)

,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


In [10]:
# MONTHLY_FX:
mfx_df <- read.csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv', colClasses=c(DATE="character"))
 head(mfx_df)

,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-01,1.319276
2,1,2017-02-01,1.310726
3,2,2017-03-01,1.338643
4,3,2017-04-01,1.344021
5,4,2017-05-01,1.360705
6,5,2017-06-01,1.329805


## Problem 2
#### Read Datasets and Load Tables
Read the datasets into R dataframes using the urls provided above. Then load your tables.


###  Solution 2


In [15]:
    sqlSave(conn, crop_df, "CROP_DATA", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
    sqlSave(conn, dfx_df, "DAILY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
    sqlSave(conn, farm_df, "FARM_PRICES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
    sqlSave(conn, mfx_df, "MONTHLY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3
#### How many records are in the farm prices dataset?


### Solution 3


In [16]:

    query <- "SELECT COUNT(*) FROM FARM_PRICES;"
    view <- sqlQuery(conn,query)
    view


,1
,<int>
1,5356


## Problem 4
#### Which geographies are included in the farm prices dataset?


### Solution 4


In [17]:
query <- "SELECT DISTINCT(GEO) FROM FARM_PRICES;"
    view <- sqlQuery(conn,query)
    view

,GEO
,<fct>
1,Alberta
2,Saskatchewan


## Problem 5
#### How many hectares of Rye were harvested in Canada in 1968?


### Solution 5


In [51]:
query <- 
"SELECT SUM(HARVESTED_AREA) AS total_harvested_area FROM CROP_DATA
WHERE GEO = 'Canada' AND
    CROP_TYPE = 'Rye' AND
    YEAR(YEAR) = 1968;"
    view <- sqlQuery(conn,query)
    view

,TOTAL_HARVESTED_AREA
,<int>
1,822300


## Problem 6
#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [32]:
query <- "SELECT CROP_TYPE, PRICE_PRERMT FROM FARM_PRICES
WHERE CROP_TYPE = 'Rye'
LIMIT 6;"
    view <- sqlQuery(conn,query)
    view

,CROP_TYPE,PRICE_PRERMT
,<fct>,<dbl>
1,Rye,100.77
2,Rye,109.75
3,Rye,95.05
4,Rye,103.46
5,Rye,96.77
6,Rye,106.38


## Problem 7
#### Which provinces grew Barley? 


### Solution 7


In [34]:
query <- "SELECT DISTINCT(CROP_TYPE), GEO FROM CROP_DATA
WHERE CROP_TYPE = 'Barley';"
    view <- sqlQuery(conn,query)
    view

,CROP_TYPE,GEO
,<fct>,<fct>
1,Barley,Alberta
2,Barley,Canada
3,Barley,Saskatchewan


## Problem 8
#### Find the first and last dates for the farm prices data.


### Solution 8


In [37]:
query <- "SELECT MIN(DATE), MAX(DATE) FROM FARM_PRICES;"
 view <- sqlQuery(conn,query)
    view

,1,2
,<fct>,<fct>
1,1985-01-01,2020-12-01


## Problem 9
#### Which crops have ever reached a farm price greater than or equal to &#0036;350 per metric tonne?


### Solution 9


In [39]:
query <- "SELECT CROP_TYPE, PRICE_PRERMT FROM FARM_PRICES
WHERE PRICE_PRERMT >=350.00;"
 view <- sqlQuery(conn,query)
    view

,CROP_TYPE,PRICE_PRERMT
,<fct>,<dbl>
1,Canola,350.04
2,Canola,363.84
3,Canola,364.36
4,Canola,354.58
5,Canola,359.99
6,Canola,350.47
7,Canola,354.05
8,Canola,354.82
9,Canola,357.37


## Problem 10
#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best? 


### Solution 10


In [49]:
query <- "SELECT CROP_TYPE, AVG_YIELD FROM CROP_DATA
WHERE YEAR(YEAR) = 2000 AND
    GEO = 'Saskatchewan'
ORDER BY AVG_YIELD DESC;"
 view <- sqlQuery(conn,query)
    view

,CROP_TYPE,AVG_YIELD
,<fct>,<int>
1,Barley,2800
2,Barley,2800
3,Barley,2800
4,Wheat,2200
5,Wheat,2200
6,Wheat,2200
7,Rye,2100
8,Rye,2100
9,Rye,2100


## Problem 11
#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000? 


### Solution 11


In [57]:
query <- "SELECT CROP_TYPE,GEO, AVG_YIELD FROM CROP_DATA
WHERE YEAR(YEAR) >= 2000
ORDER BY AVG_YIELD DESC;"
 view <- sqlQuery(conn,query)
    view

,CROP_TYPE,GEO,AVG_YIELD
,<fct>,<fct>,<int>
1,Barley,Alberta,4100
2,Barley,Alberta,4100
3,Barley,Alberta,4100
4,Barley,Alberta,4100
5,Barley,Alberta,4100
6,Barley,Alberta,4100
7,Barley,Alberta,3980
8,Barley,Alberta,3980
9,Barley,Alberta,3980


## Problem 12
#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [64]:
query<- "SELECT CROP_TYPE, SUM(HARVESTED_AREA), 
    (SELECT GEO FROM FARM_PRICES 
        WHERE GEO = 'Canada') AS Province 
FROM CROP_DATA
WHERE YEAR(YEAR) = 2020 AND
    CROP_TYPE = 'Wheat'
GROUP BY CROP_TYPE;"
  view <- sqlQuery(conn,query)
    view

,CROP_TYPE,2,PROVINCE
,<fct>,<int>,<lgl>
1,Wheat,54413700,NA


## Problem 13
#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [ ]:
query <- "SELECT "
 view <- sqlQuery(conn,query)
    view

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-01 | 0.7     | Jeff Grossman | Split Problem 1 solution cell into multiple cells, fixed minor bugs |
| 2021-03-12 | 0.6     | Jeff Grossman | Cleaned up content for production |
| 2021-03-11 | 0.5     | Jeff Grossman | Moved more advanced problems to optional honours module |
| 2021-03-10 | 0.4     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems |
| 2021-03-04 | 0.3     | Jeff Grossman | Moved some problems to a new practice lab as prep for this assignment
| 2021-03-04 | 0.2     | Jeff Grossman | Sorted problems roughly by level of difficulty and relegated more advanced ones to ungraded bonus problems  |
| 2021-02-20 | 0.1     | Jeff Grossman | Started content creation                                                                                    |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
